In [5]:
!pip install spacy
!pip install newsapi-python

!python -m spacy download en_core_web_sm
import spacy
nlp_eng = spacy.load('en_core_web_sm')

from newsapi import NewsApiClient
newsapi = NewsApiClient (api_key='cc621bf117a7465b817748010294c5e0')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [6]:
articles = []
for x in range(1,5):
 articles.append(newsapi.get_everything(q='coronavirus', language='en', from_param='2020-10-10', to='2020-10-27', sort_by='relevancy', page=x))

In [7]:
import pickle

for x in range(1,5):
  filename = 'articlesCOVID.pckl'
  pickle.dump(articles, open(filename, 'wb'))

  filename = 'articlesCOVID.pckl'
  loaded_model = pickle.load(open(filename, 'rb'))

  filepath = '/content/drive/My Drive/Covid19/articlesCOVID.pckl'
  pickle.dump(loaded_model, open(filepath, 'ab'))


In [8]:
a = articles[2]['articles']
print(a)

[{'source': {'id': 'bbc-news', 'name': 'BBC News'}, 'author': 'https://www.facebook.com/bbcnews', 'title': 'US election 2020: Harris halts travel after aide tests positive for coronavirus', 'description': 'The vice-presidential candidate was not exposed to coronavirus positive staff, the campaign said.', 'url': 'https://www.bbc.co.uk/news/election-us-2020-54559605', 'urlToImage': 'https://ichef.bbci.co.uk/news/1024/branded_news/E5F8/production/_114927885_gettyimages-1228964799-1.jpg', 'publishedAt': '2020-10-15T14:39:52Z', 'content': 'image copyrightGetty Images\r\nimage captionMr Biden and Mrs Harris campaigned in Arizona together last Thursday\r\nDemocratic vice-presidential candidate Kamala Harris will halt campaign travel until Mo… [+2549 chars]'}, {'source': {'id': 'bbc-news', 'name': 'BBC News'}, 'author': 'https://www.facebook.com/bbcnews', 'title': "Coronavirus: WHO head calls herd immunity approach 'immoral'", 'description': 'Dr Ghebreyesus said allowing coronavirus to spread 

In [9]:
articles[1]['articles'][0]

{'author': 'https://www.facebook.com/bbcnews',
 'content': 'image captionThe outbreak has intensified in the past few months in Argentina - especially in the Buenos Aires province\r\nArgentina has become the fifth country in the world to record one million conf… [+877 chars]',
 'description': 'The South American nation becomes the fifth country in the world to reach the grim milestone.',
 'publishedAt': '2020-10-20T02:07:03Z',
 'source': {'id': 'bbc-news', 'name': 'BBC News'},
 'title': "Coronavirus: Argentina's confirmed cases surpass one million",
 'url': 'https://www.bbc.co.uk/news/world-latin-america-54609384',
 'urlToImage': 'https://ichef.bbci.co.uk/news/1024/branded_news/1354B/production/_114997197_hi063904237.jpg'}

In [14]:

import pandas as pd
import collections
import string

df = pd.DataFrame(columns=['Title', 'Date', 'Desc', 'Content'])

for x in range(1,5): 
  for i, article in enumerate(articles[1]['articles']):
    title = article['title']
    date = article['publishedAt']
    description = article['description']
    content = article['content']
    data = [[title, date, description, content]]
    dados = pd.DataFrame(data, columns=['Title', 'Date', 'Desc', 'Content'])
    df = df.append(dados, ignore_index=True)
df = df.dropna()
df.head()
df




,Title,Date,Desc,Content
0,Coronavirus: Argentina's confirmed cases surpa...,2020-10-20T02:07:03Z,The South American nation becomes the fifth co...,image captionThe outbreak has intensified in t...
1,"Why False Positives Matter, Too",2020-10-25T09:00:15Z,False negatives are not the only troublesome o...,The ever-increasing availability of virtual pl...
3,Coronavirus: Europe’s tough restrictions ‘abso...,2020-10-15T15:06:57Z,The World Health Organization's warning comes ...,image captionMeasures against coronavirus are ...
5,Coronavirus: Clashes in Naples over tightening...,2020-10-24T07:55:09Z,Hundreds protested in the southern Italian cit...,image captionSmoke bombs and firecrackers were...
7,"Coronavirus Has Killed Nearly 10,000 Minks in ...",2020-10-12T16:10:00Z,Humans aren’t the only creatures at risk from ...,Humans arent the only creatures at risk from d...
...,...,...,...,...
75,Harrowing New Coronavirus Surge Rages Across A...,2020-10-24T13:45:33Z,"The U.S. set a single-day record of 82,154 new...","LiveUpdated Oct. 24, 2020, 9:31 a.m. ET\r\nOct..."
76,SUNY Oneonta President Resigns After 700 Stude...,2020-10-15T22:40:58Z,The State University of New York at Oneonta ha...,Although the university did not directly tie D...
77,Can I Safely Send My Kid to Day Care? We Asked...,2020-10-23T13:29:11Z,"As states struggle with reopening, parents are...",When it comes to transmission of Covid-19 thro...
78,The I.M.F. sees early signs of a global econom...,2020-10-13T12:30:05Z,The world finance group warns that the crisis ...,The recession is hammering both advanced and e...


In [15]:

result = []
results = []
pos_tag = ['VERB', 'NOUN', 'PROPN']
c = collections.Counter()

def get_keywords_eng(content):
  doc = nlp_eng(content)
  for token in doc:
    if (token.text in nlp_eng.Defaults.stop_words or token.text in string.punctuation):
     continue
    if (token.pos_ in pos_tag):
       result.append(token.text)
  return result


for content in df.Content.values:
    results.append([('#' + x[0]) for x in collections.Counter(get_keywords_eng(content)).most_common(5)])
df['keywords'] = results
df.to_csv(r'dataset.csv')
df





,Title,Date,Desc,Content,keywords
0,Coronavirus: Argentina's confirmed cases surpa...,2020-10-20T02:07:03Z,The South American nation becomes the fifth co...,image captionThe outbreak has intensified in t...,"[#Argentina, #image, #captionThe, #outbreak, #..."
1,"Why False Positives Matter, Too",2020-10-25T09:00:15Z,False negatives are not the only troublesome o...,The ever-increasing availability of virtual pl...,"[#Argentina, #chars, #image, #captionThe, #out..."
3,Coronavirus: Europe’s tough restrictions ‘abso...,2020-10-15T15:06:57Z,The World Health Organization's warning comes ...,image captionMeasures against coronavirus are ...,"[#chars, #image, #Argentina, #captionThe, #out..."
5,Coronavirus: Clashes in Naples over tightening...,2020-10-24T07:55:09Z,Hundreds protested in the southern Italian cit...,image captionSmoke bombs and firecrackers were...,"[#chars, #image, #Argentina, #coronavirus, #bo..."
7,"Coronavirus Has Killed Nearly 10,000 Minks in ...",2020-10-12T16:10:00Z,Humans aren’t the only creatures at risk from ...,Humans arent the only creatures at risk from d...,"[#chars, #image, #Argentina, #coronavirus, #bo..."
...,...,...,...,...,...
75,Harrowing New Coronavirus Surge Rages Across A...,2020-10-24T13:45:33Z,"The U.S. set a single-day record of 82,154 new...","LiveUpdated Oct. 24, 2020, 9:31 a.m. ET\r\nOct...","[#chars, #image, #said, #Dr., #Argentina]"
76,SUNY Oneonta President Resigns After 700 Stude...,2020-10-15T22:40:58Z,The State University of New York at Oneonta ha...,Although the university did not directly tie D...,"[#chars, #image, #Dr., #said, #outbreak]"
77,Can I Safely Send My Kid to Day Care? We Asked...,2020-10-23T13:29:11Z,"As states struggle with reopening, parents are...",When it comes to transmission of Covid-19 thro...,"[#chars, #image, #Dr., #said, #outbreak]"
78,The I.M.F. sees early signs of a global econom...,2020-10-13T12:30:05Z,The world finance group warns that the crisis ...,The recession is hammering both advanced and e...,"[#chars, #image, #Dr., #said, #outbreak]"
